In [1]:
#libraries used

import pandas
import nltk
import re
from tqdm.notebook import tqdm

In [2]:
#reading dataet

beginning = 0 #where to start using data
end = 300 #where to end using data

phishing_dataset = pandas.read_csv('verified_online.csv', nrows = end/2)

benign_dataset = pandas.read_csv('Benign_list_big_final.csv', nrows = end/2, header=None)

phishing_dataset.url.tolist()

['https://mpm.payback.de/redi?lid=5927652727930224970&amp;sid=2540887&amp;kid=521355&amp;bid=2098569&amp;c=46265&amp;keyword=&amp;sr=172&amp;clickurl=https://sanjayprinters.in/hr',
 'http://kuchkhasbate.info/wp-includes/SimplePie/HTTP/signin/myaccount/success/',
 'http://52.170.238.217/prime/mobile/',
 'http://t4lh4s4lt.site/bootstrap-3.3.7/css/membermap/register_stats.php/tma/hmpwz/?missing=1cxypvb0wh7gp77',
 'http://tiny.cc/m65gfz',
 'http://smbclm.com',
 'http://inforef.16mb.com/facebook-confirm/index.htm?entry_source=NOTIFICATIONS&amp;ext=1571050844&amp;hash=AeRIKYH6cTNQJd_v&amp;notif_id',
 'http://inforef.16mb.com/facebook-confirm/',
 'https://solicitarmultiplus.com/Bradesco/desktop/paravoce/index.php?html=11,00-49,31,10-1911,00-49,31,10-19',
 'https://www.appauthy-safebr.com/authyapphref0982349823492843289href890324-023423043href90834890234appbr.com/dispositivotoken2019327873723434/classic/index.php?34.237.113.113&amp;31/10/2019%2018:14:56',
 'http://34.239.148.242/bradesc0app893

In [3]:
#checking dataset size

print(phishing_dataset.shape)
print(benign_dataset.shape)

(150, 8)
(150, 1)


In [4]:
#parsing the urls

from urllib.parse import urlparse

phishing_urls = phishing_dataset.url.tolist()

benign_urls = benign_dataset.values.tolist()

phishing_urls_labeled = []

benign_urls_labeled = []

for url in phishing_urls:

    url = [url ,1]
    
    phishing_urls_labeled.append(url)
    
for url in benign_urls:
    
    url = [url[0] ,0]
    
    benign_urls_labeled.append(url)
    
phishing_urls_labeled.extend(benign_urls_labeled)

urls = phishing_urls_labeled
parsed_urls = []

for url in tqdm(urls):
    
    url = [urlparse(url[0]),url[1]]
    
    parsed_urls.append(url)

print(parsed_urls)


[[ParseResult(scheme='https', netloc='mpm.payback.de', path='/redi', params='', query='lid=5927652727930224970&amp;sid=2540887&amp;kid=521355&amp;bid=2098569&amp;c=46265&amp;keyword=&amp;sr=172&amp;clickurl=https://sanjayprinters.in/hr', fragment=''), 1], [ParseResult(scheme='http', netloc='kuchkhasbate.info', path='/wp-includes/SimplePie/HTTP/signin/myaccount/success/', params='', query='', fragment=''), 1], [ParseResult(scheme='http', netloc='52.170.238.217', path='/prime/mobile/', params='', query='', fragment=''), 1], [ParseResult(scheme='http', netloc='t4lh4s4lt.site', path='/bootstrap-3.3.7/css/membermap/register_stats.php/tma/hmpwz/', params='', query='missing=1cxypvb0wh7gp77', fragment=''), 1], [ParseResult(scheme='http', netloc='tiny.cc', path='/m65gfz', params='', query='', fragment=''), 1], [ParseResult(scheme='http', netloc='smbclm.com', path='', params='', query='', fragment=''), 1], [ParseResult(scheme='http', netloc='inforef.16mb.com', path='/facebook-confirm/index.htm'

In [5]:
#separaing in paths and domains

registered_domains = []
paths = []

for parsed_url in tqdm(parsed_urls):
    
    if parsed_url[1] == 1:
        registered_domains.append([parsed_url[0].netloc,1])
        paths.append([parsed_url[0].path,1])
    else:
        registered_domains.append([parsed_url[0].netloc,0])
        paths.append([parsed_url[0].path,0])

splitted_paths =[]

for path in tqdm(paths):
    
    split_path = path[0].split('/')
    size_split_path = len(split_path) - 1
    
    if path[1] == 1:
    
        splitted_paths.append([split_path[1:size_split_path],1])
    
    if path[1] == 0:

        splitted_paths.append([split_path[1:size_split_path],0])
    
print(splitted_paths)


[[[], 1], [['wp-includes', 'SimplePie', 'HTTP', 'signin', 'myaccount', 'success'], 1], [['prime', 'mobile'], 1], [['bootstrap-3.3.7', 'css', 'membermap', 'register_stats.php', 'tma', 'hmpwz'], 1], [[], 1], [[], 1], [['facebook-confirm'], 1], [['facebook-confirm'], 1], [['Bradesco', 'desktop', 'paravoce'], 1], [['authyapphref0982349823492843289href890324-023423043href90834890234appbr.com', 'dispositivotoken2019327873723434', 'classic'], 1], [['bradesc0app8934908893893429href0923489723498234href9083420894234324appbr.com'], 1], [['authyapphref0982349823492843289href890324-023423043href90834890234appbr.com', 'dispositivotoken2019327873723434', 'classic'], 1], [['facebook-confirm'], 1], [['facebook-confirm'], 1], [['facebook-confirm'], 1], [['facebook-confirm'], 1], [['facebook-confirm'], 1], [['facebook-confirm'], 1], [[], 1], [[], 1], [['acessoautenticosax.com', 'Digital', 'Produtos'], 1], [[], 1], [['c34e0e02eb881e6728224fbee2b6c96b', 'a420d057e10f146fed117277ceeb13a3'], 1], [['c34e0e02

In [6]:
##Seperating words in readable

from math import log

# Build a cost dictionary, assuming Zipf's law and cost = -math.log(probability).
words = open("words-by-frequency.txt").read().split()
wordcost = dict((k, log((i+1)*log(len(words)))) for i,k in enumerate(words))
maxword = max(len(x) for x in words)

def infer_spaces(s):
    """Uses dynamic programming to infer the location of spaces in a string
    without spaces."""

    # Find the best match for the i first characters, assuming cost has
    # been built for the i-1 first characters.
    # Returns a pair (match_cost, match_length).
    def best_match(i):
        candidates = enumerate(reversed(cost[max(0, i-maxword):i]))
        return min((c + wordcost.get(s[i-k-1:i], 9e999), k+1) for k,c in candidates)

    # Build the cost array.
    cost = [0]
    for i in range(1,len(s)+1):
        c,k = best_match(i)
        cost.append(c)

    # Backtrack to recover the minimal-cost string.
    out = []
    i = len(s)
    while i>0:
        c,k = best_match(i)
        assert c == cost[i]
        out.append(s[i-k:i])
        i -= k

    return " ".join(reversed(out))

In [7]:
#clean domains

cleaned_registered_domains = []

for domain in tqdm(registered_domains):
    domain[0] = re.sub("\d", "", domain[0]) #remove numbers
    domain[0] = re.sub("[-]", "", domain[0]) #remove -
    domain[0] = re.sub("\.[^.]*$", "", domain[0]) #remove evreything after last dot, basically the .com, etc
    domain[0] = domain[0].replace('www.','') #remove www.
    domain[0] = domain[0].replace('.','') #remove .
    domain[0] = domain[0].replace('_','') #remove _
    domain[0] = domain[0].lower() #make all wdomain[0]ords lowercase
    domain[0] = infer_spaces(domain[0]).split(" ") #seperate in words
    for word in domain[0]: #remove single letters
        if len(word) == 1:
            domain[0].remove(word)
    cleaned_registered_domains.append(domain)
cleaned_registered_domains

[[['pm', 'pay', 'back'], 1],
 [['ku', 'ch', 'khas', 'bate'], 1],
 [[''], 1],
 [['lhs', 'lt'], 1],
 [['tiny'], 1],
 [['mbc', 'lm'], 1],
 [['in', 'for', 'ef', 'mb'], 1],
 [['in', 'for', 'ef', 'mb'], 1],
 [['solicit', 'ar', 'multi', 'plus'], 1],
 [['appa', 'thy', 'safe', 'br'], 1],
 [[''], 1],
 [['appa', 'thy', 'safe', 'br'], 1],
 [['ref', 'sec', 'mb'], 1],
 [['ref', 'sec', 'mb'], 1],
 [['sec', 'smb'], 1],
 [['sec', 'smb'], 1],
 [['section', 'pf', 'mb'], 1],
 [['section', 'pf', 'mb'], 1],
 [['arc', 're', 'pro', 'web', 'host', 'app'], 1],
 [['is', 'sbk', 'muf', 'in', 'kinj'], 1],
 [['secure', 'secure', 'web', 'session'], 1],
 [['client', 'co', 'mumbler'], 1],
 [['x', 'sph'], 1],
 [['x', 'sph'], 1],
 [['appa', 'thy', 'safe', 'br'], 1],
 [['tesla'], 1],
 [['ds', 'gn', 'pool'], 1],
 [['of', 'us', 'even', 'ment', 'draco', 'niro'], 1],
 [['client', 'co', 'mumbler'], 1],
 [['my', 'billing', 'login'], 1],
 [['audio', 'booked'], 1],
 [['sad'], 1],
 [['ud', 'inca', 'ko', 'web', 'host', 'app'], 1],


In [8]:
#clean paths

cleaned_splitted_paths = []

for splitted_path in tqdm(splitted_paths):
    
    cleaned_splitted_path = []
    
    for path in splitted_path[0]:
        path = re.sub("\d", "", path) #remove numbers
        path = path.replace('-', '') #remove -
        path = path.replace('.','') #remove .
        path = path.replace('_','') #remove _
        path = path.lower() #make all words lowercase
        path = infer_spaces(path).split(" ") #seperate in words
        
        if path != []: #if not empty
            
            temporary_path = path.copy()
            
            for word in path: #remove single letters
                if len(word) == 1:
                    temporary_path.remove(word)
            path = temporary_path
        cleaned_splitted_path.append(path)
        
    cleaned_splitted_paths.append([cleaned_splitted_path,splitted_path[1]])

cleaned_splitted_paths_flatted = []
    
for l in cleaned_splitted_paths:
    flat_list = [item for sublist in l[0] for item in sublist] #flatten list
    flat_list = list(set(flat_list)) #remove duplicates
    cleaned_splitted_paths_flatted.append([flat_list,l[1]])

cleaned_splitted_paths = cleaned_splitted_paths_flatted 
    
cleaned_splitted_paths

[[[], 1],
 [['in',
   'sign',
   'http',
   'success',
   'wp',
   'my',
   'includes',
   'account',
   'simple',
   'pie'],
  1],
 [['prime', 'mobile'], 1],
 [['css',
   'stats',
   'pw',
   'bootstrap',
   'php',
   'map',
   'register',
   'member',
   'tm',
   'hm'],
  1],
 [[], 1],
 [[], 1],
 [['facebook', 'confirm'], 1],
 [['facebook', 'confirm'], 1],
 [['desktop', 'es', 'voce', 'co', 'para', 'brad'], 1],
 [['is',
   'href',
   'app',
   'com',
   'br',
   'classic',
   'positiv',
   'au',
   'thy',
   'token'],
  1],
 [['href', 'app', 'com', 'escap', 'br', 'ph', 'brad', 'ref'], 1],
 [['is',
   'href',
   'app',
   'com',
   'br',
   'classic',
   'positiv',
   'au',
   'thy',
   'token'],
  1],
 [['facebook', 'confirm'], 1],
 [['facebook', 'confirm'], 1],
 [['facebook', 'confirm'], 1],
 [['facebook', 'confirm'], 1],
 [['facebook', 'confirm'], 1],
 [['facebook', 'confirm'], 1],
 [[], 1],
 [[], 1],
 [['pro', 'aute', 'du', 'digital', 'com', 'to', 'sax', 'ntico', 'so', 'aces'],
  1

In [ ]:
#Get trends for domains

import sys
from pytrends.request import TrendReq
import time

pytrends = TrendReq(
    retries = 5,
)

domains_related_queries = []

size = end - beginning
# size = 1

with tqdm(total = size, file=sys.stdout) as pbar:
    
    iteration = beginning
    
    while iteration < end:
        domain = cleaned_registered_domains[iteration][0]
#         print(domain)
        try:
            pytrends.build_payload(kw_list=domain[0:4])
            dictionairy_related_query = pytrends.related_queries()
            
            related_queries = []

            for word in domain:
                related = dictionairy_related_query[word]['top']['query'].to_list()
                related_queries.extend(related)
            domains_related_queries.append([related_queries,cleaned_registered_domains[iteration][1]])
            iteration = iteration + 1
            pbar.set_description('processed: %d' % (1 + iteration))
            pbar.update(1)
        except:
            time.sleep(10)
            print('what')
            if domain == ['']:
                iteration = iteration + 1
                domains_related_queries.append([[''],cleaned_registered_domains[iteration-1][0]])
                pbar.set_description('processed: %d' % (1 + iteration))
                pbar.update(1)

# for domain in tqdm(cleaned_registered_domains[beginning:end], desc = 'related query loop'):
    
#     try:
#         pytrends.build_payload(kw_list=domain[0])
#         dictionairy_related_query = pytrends.related_queries()
        
#         related_queries = []
        
#         for word in domain[0]:
#             related = dictionairy_related_query[word]['top']['query'].to_list()
#             related_queries.extend(related)
#         domains_related_queries.append([related_queries,domain[1]])
    
#     except:
#         pass

what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what
what


In [20]:
print(domain)

['arc', 're', 'pro', 'web', 'host', 'app']


In [18]:
splitted_domains_related_queries = []

for domain_related in domains_related_queries:
    
    words = []
    
    for word in domain_related[0]:
        
        splitted_word = word.split()
        words.extend(splitted_word)
    
    splitted_domains_related_queries.append([words,domain_related[1]])

In [12]:
print(cleaned_splitted_paths)

[[[], 1], [['http', 'success', 'simple', 'wp', 'pie', 'includes', 'my', 'in', 'account', 'sign'], 1], [['prime', 'mobile'], 1], [['map', 'tm', 'bootstrap', 'register', 'member', 'hm', 'pw', 'css', 'stats', 'php'], 1], [[], 1], [[], 1], [['confirm', 'facebook'], 1], [['confirm', 'facebook'], 1], [['voce', 'co', 'es', 'para', 'brad', 'desktop'], 1], [['positiv', 'au', 'thy', 'app', 'is', 'com', 'token', 'href', 'classic', 'br'], 1], [['app', 'com', 'ref', 'escap', 'brad', 'href', 'ph', 'br'], 1], [['positiv', 'au', 'thy', 'app', 'is', 'com', 'token', 'href', 'classic', 'br'], 1], [['confirm', 'facebook'], 1], [['confirm', 'facebook'], 1], [['confirm', 'facebook'], 1], [['confirm', 'facebook'], 1], [['confirm', 'facebook'], 1], [['confirm', 'facebook'], 1], [[], 1], [[], 1], [['ntico', 'com', 'pro', 'digital', 'sax', 'to', 'du', 'so', 'aute', 'aces'], 1], [[], 1], [['ebc', 'def', 'fbe', 'eee', 'ceeb', 'fed', 'be'], 1], [['ebc', 'def', 'fbe', 'eee', 'ceeb', 'fed', 'be', 'as'], 1], [['posit

In [16]:
#Get trends for paths

#Get trends for paths

import sys
from pytrends.request import TrendReq
import time


pytrends = TrendReq(
    retries = 5,
)

paths_related_queries = []

size = end - beginning

paths_related_queries = []

with tqdm(total = size, file=sys.stdout) as pbar:
    
    iteration = beginning
    
    path_related_queries = []
    
    while iteration<end:
        
        paths = cleaned_splitted_paths[iteration][0]
        print(paths)
        
        if paths!=[]:
            for path in paths:

                try:

                    pytrends.build_payload(kw_list=[path])

                    dictionairy_related_queries = pytrends.related_queries()

                    related = dictionairy_related_queries[path]['top']['query'].to_list()

                    path_related_queries.extend(related)

                    iteration = iteration + 1

                    pbar.set_description('processed: %d' % (1 + iteration))

                    pbar.update(1)

                except:

                    time.sleep(0.5)
                    if path == '':
                        iteration = iteration+1
                        path_related_queries.append([[''],paths_related_query[iteration]])
                        pbar.set_description('processed: %d' % (1 + iteration))
                        pbar.update(1)

            paths_related_queries.append([path_related_queries,cleaned_splitted_paths[iteration][1]])
        
        else:
            paths_related_queries.append([[''],cleaned_splitted_paths[iteration][1]])
            
            iteration = iteration + 1

            pbar.set_description('processed: %d' % (1 + iteration))

            pbar.update(1)
# for paths in tqdm(cleaned_splitted_paths, desc = 'related query loop'):
    
#     try:
#         pytrends.build_payload(kw_list=paths)
#         dictionairy_related_query = pytrends.related_queries()

#         related_queries = []

#         for path in paths:
#             related_queries.append(related_query)
            
#         paths_related_queries.append(related_queries)
    
#     except:
#         time.sleep(0.01) #google trends only allows 10 per minute
        
    
    
        
# paths_related_queries

# from pytrends.request import TrendReq
# import time

# pytrends = TrendReq(
#     retries = 5,
# )

# paths_related_queries = []

# for paths in tqdm(cleaned_splitted_paths[beginning:end], desc = 'related query loop'):
    
#     path_related_queries = []
    
#     for path in paths[0]:
        
#         try:
            
#             pytrends.build_payload(kw_list=[path])
            
#             dictionairy_related_queries = pytrends.related_queries()
            
#             related = dictionairy_related_queries[path]['top']['query'].to_list()
            
#             path_related_queries.extend(related)
            
#         except:
            
#             time.sleep(0.01) #google trends only allows 10 per minute
        
#     paths_related_queries.append([path_related_queries,paths[1]])

[]
['http', 'success', 'simple', 'wp', 'pie', 'includes', 'my', 'in', 'account', 'sign']
['http', 'success', 'simple', 'wp', 'pie', 'includes', 'my', 'in', 'account', 'sign']
['http', 'success', 'simple', 'wp', 'pie', 'includes', 'my', 'in', 'account', 'sign']
['http', 'success', 'simple', 'wp', 'pie', 'includes', 'my', 'in', 'account', 'sign']
['http', 'success', 'simple', 'wp', 'pie', 'includes', 'my', 'in', 'account', 'sign']
['http', 'success', 'simple', 'wp', 'pie', 'includes', 'my', 'in', 'account', 'sign']
['http', 'success', 'simple', 'wp', 'pie', 'includes', 'my', 'in', 'account', 'sign']
['http', 'success', 'simple', 'wp', 'pie', 'includes', 'my', 'in', 'account', 'sign']
['http', 'success', 'simple', 'wp', 'pie', 'includes', 'my', 'in', 'account', 'sign']
['http', 'success', 'simple', 'wp', 'pie', 'includes', 'my', 'in', 'account', 'sign']
['http', 'success', 'simple', 'wp', 'pie', 'includes', 'my', 'in', 'account', 'sign']
['http', 'success', 'simple', 'wp', 'pie', 'include

KeyboardInterrupt: 

In [17]:
paths_related_queries

[[[], 1],
 [['my love',
   'my life',
   'my account',
   'google my',
   'my facebook',
   'my friend',
   'my ip',
   'my girl',
   'my location',
   'my way',
   'my house',
   'my little pony',
   'my video',
   'my pay',
   'make my trip',
   'my school',
   'book my show',
   'my email',
   'my family',
   'what is my ip',
   'my mail',
   'rate my professor',
   'my days',
   'my business',
   'find my iphone'],
  1],
 [[], 1],
 [[], 1],
 [[], 1],
 [[], 1],
 [[], 1],
 [['login facebook',
   'facebook entrar',
   'facebook iniciar',
   'facebook iniciar sesion',
   'facebook log in',
   'facebook español',
   'facebook download',
   'facebook en español',
   'facebook com',
   'my facebook',
   'free facebook',
   'facebook sign in',
   'facebook login in',
   'mi facebook',
   'facebook lite',
   'facebook messenger',
   'www facebook',
   'facebook account',
   'entrar no facebook',
   'facebook mobile',
   'dang nhap facebook',
   'facebook connexion',
   'fb login facebook',


In [18]:
splitted_paths_related_queries = []

for path_related in paths_related_queries:
    
    words = []
    
    for word in path_related[0]:
        
        splitted_word = word.split()
        words.extend(splitted_word)
    
    splitted_paths_related_queries.append([words,path_related[1]])
    

splitted_paths_related_queries

[[[], 1],
 [['my',
   'love',
   'my',
   'life',
   'my',
   'account',
   'google',
   'my',
   'my',
   'facebook',
   'my',
   'friend',
   'my',
   'ip',
   'my',
   'girl',
   'my',
   'location',
   'my',
   'way',
   'my',
   'house',
   'my',
   'little',
   'pony',
   'my',
   'video',
   'my',
   'pay',
   'make',
   'my',
   'trip',
   'my',
   'school',
   'book',
   'my',
   'show',
   'my',
   'email',
   'my',
   'family',
   'what',
   'is',
   'my',
   'ip',
   'my',
   'mail',
   'rate',
   'my',
   'professor',
   'my',
   'days',
   'my',
   'business',
   'find',
   'my',
   'iphone'],
  1],
 [[], 1],
 [[], 1],
 [[], 1],
 [[], 1],
 [[], 1],
 [['login',
   'facebook',
   'facebook',
   'entrar',
   'facebook',
   'iniciar',
   'facebook',
   'iniciar',
   'sesion',
   'facebook',
   'log',
   'in',
   'facebook',
   'español',
   'facebook',
   'download',
   'facebook',
   'en',
   'español',
   'facebook',
   'com',
   'my',
   'facebook',
   'free',
   'facebook

In [28]:
df_paths_related = pandas.DataFrame(splitted_paths_related_queries)

df_domains_related = pandas.DataFrame(splitted_domains_related_queries)
try:
    df_paths_exist_related = pandas.read_csv('related_paths.csv', header=None).append(df_paths_related, ignore_index=True)

    df_domains_exist_related = pandas.read_csv('related_domains.csv', header=None).append(df_domains_related, ignore_index=True)
    
except:
    pass

In [29]:
df_domains_related

,0,1
0,"[am, pm, am, pm, time, pm, modi, concurso, pm,...",1
1,"[lagu, download, lagu, lirik, lagu, ku, klux, ...",1
2,"[chrysler, lhs, chrysler, lhs, school, lhs, hi...",1
3,"[tiny, house, tiny, houses, tiny, homes, tiny,...",1
4,"[تردد, mbc, bollywood, mbc, bollywood, mbc, li...",1
...,...,...
433,"[hindi, bhojpuri, video, gana, sexy, photo, fo...",0
434,"[hindi, bhojpuri, video, gana, sexy, photo, fo...",0
435,"[hindi, bhojpuri, video, gana, sexy, photo, fo...",0
436,"[hindi, bhojpuri, video, gana, sexy, photo, fo...",0


In [30]:
try:
    df_paths_exist_related.to_csv(r'related_paths.csv',index=False, header=False)
    df_domains_exist_related.to_csv(r'related_domains.csv',index=False, header=False)
    print("File already existed")
except:
    df_domains_related.to_csv(r'related_domains.csv',index=False, header=False)
    df_paths_related.to_csv(r'related_paths.csv',index=False, header=False)
    print("Created a new file")

File already existed


In [31]:
df_paths = pandas.DataFrame(cleaned_splitted_paths)

df_domains = pandas.DataFrame(cleaned_registered_domains)

try:
    df_paths_exist = pandas.read_csv('paths.csv', header=None).append(df_paths, ignore_index=True)

    df_domains_exist = pandas.read_csv('domains.csv', header=None).append(df_domains, ignore_index=True)

except:
    pass

In [32]:
try:
    df_paths_exist.to_csv(r'paths.csv',index=False, header=False)
    df_domains_exist.to_csv(r'domains.csv',index=False, header=False)
    print("File already existed")
except:
    df_domains.to_csv(r'domains.csv',index=False, header=False)
    df_paths.to_csv(r'paths.csv',index=False, header=False)
    print("Created a new file")

Created a new file
